In [59]:
import glob
import os
import numpy as np

In [1]:
data = np.load('../data/segmented_data_300.npy')
data.shape

X = data[:,:,:-1]
Y = data[:,:,-1]
Y = Y.max(axis=1)

n_channels = X.shape[2]

X.shape, Y.shape,n_channels


NameError: name 'np' is not defined

In [ ]:

X = data[:,:,:-1]
Y = data[:,:,-1]
Y = Y.max(axis=1)

n_channels = X.shape[2]

X.shape, Y.shape,n_channels


In [61]:
# # Normalize
# X = X / np.max(np.abs(X), axis=1)[:,None,:]

# # Shuffle
# np.random.seed(1)
# shuffle = np.random.choice(np.arange(len(Y)), size=len(Y), replace=False)
# X = X[shuffle]
# Y = Y[shuffle]

In [62]:
np.isnan(X).sum()

0

In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.Input(shape=(X.shape[1], X.shape[2])))
model.add(layers.LSTM(64))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

In [64]:
import numpy
from keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


class RocCallback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = model.predict(self.x)
        roc_train = roc_auc_score(self.y, y_pred_train)
        y_pred_val = model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('roc-auc_train: ', roc_train)
        print('roc-auc_val: ', roc_val)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

# Function that trains the model
def train_model(X, Y):
    X = X.reshape(X.shape[0], -1, n_channels)   # reshape DH table to 3d numpy array
    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, stratify=Y, test_size = 0.1)
    roc = RocCallback(training_data=(X_train, Y_train), validation_data=(X_valid, Y_valid))
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
    # model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), callbacks=[roc], batch_size = 200, epochs=100)
    model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), callbacks=[roc], batch_size = 200, epochs=100)

# Function that gets the model's predictions on input data
def predict_with_model(X):
    X = X.reshape(X.shape[0], -1, n_channels)  # reshape DH table to 3d numpy array
    Y_pred = model.predict(X, batch_size=200)
    return Y_pred

# Function to extract a list element at a given index
def get_predicted_class(data, idx):
    return data[idx]

# Split the data into training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size = 0.2)

# Convert numpy arrays X_train and Y_train to DH table X_table
n_rows = X_train.shape[0]
n_cols = X_train.shape[1] * X_train.shape[2]
column_names = ['Col_'+str(i) for i in range(n_cols)]
X_reshaped = X_train.reshape(n_rows, n_cols)
# X_table = numpy.to_table(X_reshaped, cols=column_names)

def add_class_col(index):
    y_class = [int(i) for i in Y_train.tolist()]
    return y_class[index]

# X_table = X_table.update(["Class = (int)add_class_col(i)"])


train_model(X_train, Y_train)

# # Train the model
# learn.learn(
#     table=X_table,
#     model_func=train_model,
#     inputs=[learn.Input(column_names, table_to_array_double), learn.Input(["Class"], table_to_array_int)],
#     outputs=None,
#     batch_size=200
# )


# Convert numpy array X_test to DH table X_table_test
X_reshaped_test = X_test.reshape(X_test.shape[0], n_cols)
# X_table_test = numpy.to_table(X_reshaped_test, cols=column_names)

# Use the learn function to create a new table that contains predicted values
y_pred = predict_with_model(X_test)

# predicted = learn.learn(
#     table=X_table_test,
#     model_func=predict_with_model,
#     inputs=[learn.Input(column_names, table_to_array_double)],
#     outputs=[learn.Output("PredictedClass", get_predicted_class, "int")],
#     batch_size=200
# )


Epoch 1/100
2/2 [==============================] - 0s 11ms/step
roc-auc_train:  0.5384615384615384
roc-auc_val:  0.6733333333333333
2/2 [==============================] - 6s 2s/step - loss: 0.7731 - accuracy: 0.4777 - val_loss: 0.5960 - val_accuracy: 0.8571
Epoch 2/100
2/2 [==============================] - 0s 11ms/step
roc-auc_train:  0.5653980166175289
roc-auc_val:  0.7866666666666666
2/2 [==============================] - 1s 570ms/step - loss: 0.7153 - accuracy: 0.5924 - val_loss: 0.5937 - val_accuracy: 0.8571
Epoch 3/100
2/2 [==============================] - 0s 12ms/step
roc-auc_train:  0.6035915304207987
roc-auc_val:  0.5933333333333334
2/2 [==============================] - 1s 444ms/step - loss: 0.6801 - accuracy: 0.6624 - val_loss: 0.5939 - val_accuracy: 0.8571
Epoch 4/100
2/2 [==============================] - 0s 11ms/step
roc-auc_train:  0.6152952738318591
roc-auc_val:  0.6400000000000001
2/2 [==============================] - 1s 514ms/step - loss: 0.6702 - accuracy: 0.6401 -

In [70]:
y_pred = predict_with_model(X_test)
y_test = Y_test

from sklearn.metrics import precision_score, recall_score
y_pred_c = y_pred.round().astype(int)
precision = precision_score(y_test, y_pred_c)
recall = recall_score(y_test, y_pred_c)

loss, accuracy = model.evaluate(X_test, y_test)

print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

3/3 [==============================] - 0s 15ms/step - loss: 0.4142 - accuracy: 0.8750
Loss: 0.41420790553092957
Accuracy: 0.875
Precision: 0.0
Recall: 0.0


/home/weasel/reps/ai-seizure-detectives/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
y_pred_c.sum(), y_test.sum(), y_test.shape

(0, 11.0, (88,))

In [69]:
(y_pred_c[:,0].astype(bool) & y_test.astype(bool)).sum()

0